In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss, confusion_matrix 
from sklearn.tree import tree

In [2]:
true = pd.read_csv(r"True.csv")
fake = pd.read_csv(r"Fake.csv")

In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [20]:
true

,title,text,subject,date,category
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [21]:
fake

,title,text,subject,date,category
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [4]:
true['category'] = 1
fake['category'] = 0

In [5]:
true.head()

,title,text,subject,date,category
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [6]:
combined = pd.concat([true,fake])
combined

,title,text,subject,date,category
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [7]:
combined['text'] = combined['title']+" "+combined['text']

del combined['title']
del combined['subject']
del combined['date']

In [8]:
combined

,text,category
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1
...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,0
23479,How to Blow $700 Million: Al Jazeera America F...,0


In [9]:
X = combined['text']
y = combined['category']

In [10]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(X)
frequency_matrix = count_vectorizer.transform(X)

In [11]:
tfidf = TfidfTransformer()
tfidf.fit(frequency_matrix)
tf_idf_matrix = tfidf.fit_transform(frequency_matrix)
print(tf_idf_matrix.shape)
print(tf_idf_matrix)

(44898, 122513)
  (0, 121215)	0.025645778048507702
  (0, 121196)	0.013623517314131268
  (0, 120924)	0.021363230563101
  (0, 120806)	0.048497738236745934
  (0, 119630)	0.03363366907873959
  (0, 119510)	0.01902934823029519
  (0, 119114)	0.033228167635568635
  (0, 118765)	0.029268705361287294
  (0, 118719)	0.08026718999024494
  (0, 118683)	0.05201025706857723
  (0, 118487)	0.039943504130049295
  (0, 118399)	0.016204010796951707
  (0, 118327)	0.030149462611413637
  (0, 118321)	0.059273010965436856
  (0, 118300)	0.03307638890050219
  (0, 118294)	0.012469316524035262
  (0, 118012)	0.021912072599917766
  (0, 117943)	0.017303841892419982
  (0, 117941)	0.037143754983294364
  (0, 117787)	0.029479795319164195
  (0, 117768)	0.037609759956050585
  (0, 117638)	0.05637208284565035
  (0, 117525)	0.01708805899434997
  (0, 117335)	0.03079476191678583
  (0, 117316)	0.017890625734549574
  :	:
  (44897, 11929)	0.021227614081245134
  (44897, 11679)	0.037646249099555856
  (44897, 11326)	0.009244119725762539


In [12]:
X_train, X_test, y_train, y_test = train_test_split(tf_idf_matrix, y, test_size=0.2, random_state=42)

In [13]:
X_train1, X_val, y_train1, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state=42)

In [14]:
logistic_reg = LogisticRegression()
logistic_reg.fit(X_train1, y_train1)

y_pred_val = logistic_reg.predict(X_val)
y_pred_train1 = logistic_reg.predict(X_train1)
y_pred_test = logistic_reg.predict(X_test)

acc_val = log_loss(y_val, y_pred_val)
acc_train1 = log_loss(y_train1, y_pred_train1)
acc_test = log_loss(y_test, y_pred_test)
print("Validation error: ",1-acc_val)
print("Traning error: ",1-acc_train1)
print("Test error: ",1-acc_test)
print("\n")
score_val = logistic_reg.score(X_val, y_val)
score_train1 = logistic_reg.score(X_train1, y_train1)
score_test = logistic_reg.score(X_test, y_test)
print("Validation score: ",score_val)
print("Traning score: ",score_train1)
print("Test score: ",score_test)

Validation error:  0.47434683658955534
Traning error:  0.6895289072259151
Test error:  0.5653740642722995


Validation score:  0.9847809948032665
Traning score:  0.9910110571951316
Test score:  0.9874164810690423


In [15]:
conf_mat = confusion_matrix(y_test,y_pred_test)

In [18]:
precision = conf_mat[1,1]/(conf_mat[1,1]+conf_mat[0,1])
print('The precision of the logistic model is: ',precision)
recall = conf_mat[1,1]/(conf_mat[1,1]+conf_mat[1,0])
f1 = 2*((precision*recall)/(precision+recall))
print('The F1 score of the logistic model is: ',1-f1)

The precision of the logistic model is:  0.9830469644902634
The F1 score of the logistic model is:  0.01299597469810243


In [31]:
depth_len=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
for i in depth_len:
    dtc = tree.DecisionTreeClassifier(max_depth = i)
    dtc.fit(X_train1, y_train1)
    y_pred_train1 = dtc.predict(X_train1)
    y_pred_val = dtc.predict(X_val)
    y_pred_test = dtc.predict(X_test)
    acc_train1 = accuracy_score(y_train1, y_pred_train1)
    acc_val = accuracy_score(y_val, y_pred_val)
    acc_test = accuracy_score(y_test, y_pred_test)
    print("Traning error for depth length ",i," : ", 1-acc_train1)
    print("Validation error for depth length ",i," : ", 1-acc_val)
    print("Test error for depth length ",i," : ", 1-acc_test)
    conf_mat_dtc = confusion_matrix(y_test,y_pred_test)
    precision_dtc = conf_mat_dtc[1,1]/(conf_mat_dtc[1,1]+conf_mat_dtc[0,1])
    recall_dtc = conf_mat_dtc[1,1]/(conf_mat_dtc[1,1]+conf_mat_dtc[1,0])
    f1_dtc = 2*((precision_dtc*recall_dtc)/(precision_dtc+recall_dtc))
    print('The F1 score of the Decision tree model is: ',f1_dtc)
    print("\n")

Traning error for depth length  1  :  0.007716172142232081
Validation error for depth length  1  :  0.006867112100965134
Test error for depth length  1  :  0.0075723830734966135
The F1 score of the Decision tree model is:  0.9921875


Traning error for depth length  2  :  0.006164982897144178
Validation error for depth length  2  :  0.005753526354862704
Test error for depth length  2  :  0.006236080178173675
The F1 score of the Decision tree model is:  0.9935572940635068


Traning error for depth length  3  :  0.00509108265054492
Validation error for depth length  3  :  0.005103934669636212
Test error for depth length  3  :  0.00567928730512246
The F1 score of the Decision tree model is:  0.9941305098400276


Traning error for depth length  4  :  0.005051308567337487
Validation error for depth length  4  :  0.0051967334818113775
Test error for depth length  4  :  0.00567928730512246
The F1 score of the Decision tree model is:  0.9941305098400276


Traning error for depth length  5  :  